아래의 code는 colab TPU MNIST model에서 필요한 부분만 가져온 것이다.
이는 이전 session 에서 정의된 MNIST model 에서 단순히 network의 크기만 늘린것이다.

Network를 deep하게 가져가는 이유는 단순히 TPU가 얼마나 빠른지 test하기 목적 밖에 없다.

따라서 아래의 code를 "None" mode 와 "GPU" mode 에서 실행 시켜 보기 바란다.
속도의 차이가 어느 정도 나는지 check 해 보자.

그리고 이결과를 TPU 의 계산속도와 비교해 보자.

아래의 예제 code에서는 for 문을 time 을 가지고 제어하지는 않는다.
아직은 복잡한 문제가 아니기 때문에 time을 가지고 제어하는 것은 향후 복잡한 문제에 들어갈때 몇가지 예제를 만들 예정인다.


In [1]:
import tensorflow as tf
import numpy as np

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# add empty color dimension
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

def create_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
    model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(tf.keras.layers.Dropout(0.25))

    model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
    model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.25))

    model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
    model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(tf.keras.layers.Dropout(0.25))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.Activation('elu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10))
    model.add(tf.keras.layers.Activation('softmax'))
    return model

model = create_model()
model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, ),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'])

model.fit(
    x_train.astype(np.float32), y_train.astype(np.float32),
    epochs=17,
    steps_per_epoch=60,
    validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
    validation_freq=17
)

model.save_weights('./fashion_mnist.h5', overwrite=True)

4423680/4422102 [==============================] - 0s 0us/step
Epoch 1/17
60/60 [==============================] - 4s 59ms/step - loss: 1.2366 - sparse_categorical_accuracy: 0.6655
Epoch 2/17
60/60 [==============================] - 4s 59ms/step - loss: 0.5479 - sparse_categorical_accuracy: 0.8135
Epoch 3/17
60/60 [==============================] - 4s 59ms/step - loss: 0.4485 - sparse_categorical_accuracy: 0.8448
Epoch 4/17
60/60 [==============================] - 4s 60ms/step - loss: 0.3850 - sparse_categorical_accuracy: 0.8659
Epoch 5/17
60/60 [==============================] - 4s 60ms/step - loss: 0.3477 - sparse_categorical_accuracy: 0.8778
Epoch 6/17
60/60 [==============================] - 4s 60ms/step - loss: 0.3204 - sparse_categorical_accuracy: 0.8880
Epoch 7/17
60/60 [==============================] - 4s 60ms/step - loss: 0.2897 - sparse_categorical_accuracy: 0.8962
Epoch 8/17
60/60 [==============================] - 4s 59ms/step - loss: 0.2722 - sparse_categorical_accuracy: 